This is pbaylies' StyleGAN Encoder and tutorial, ported to StyleGAN 2.
  
 * Note: make sure you're using the GPU runtime for this - see Runtime -> Change Runtime Type in Google Colab

In [ ]:
!pip install --force tqdm==4.24.0
%tensorflow_version 1.x

First, clone the repo:

In [ ]:
!git clone https://github.com/PaterAeneas/stylegan2encoder
%cd stylegan2encoder
!curl -O https://raw.githubusercontent.com/pbaylies/stylegan-encoder/master/config.py 

TODO: Optionally, try training a ResNet of your own if you like; this could take a while. Remember, there's a pre-trained model linked in the repo that works with the FFHQ faces StyleGAN model)

In [ ]:
#!python train_resnet.py --help
#!python train_resnet.py --test_size 256 --batch_size 1024 --loop 1 --max_patience 1

# Upload finetuned_resnet.h5 to colab storage
# !mv /content/finetuned_resnet.h5 /content/stylegan2encoder/data

Next, let's get some test images to work with... These are from Pexels, which has free stock photos.

In [ ]:
!mkdir aligned_images raw_images
!wget -O raw_images/stock_photo1.jpg 'https://images.pexels.com/photos/614810/pexels-photo-614810.jpeg?auto=compress&cs=tinysrgb&dpr=3&h=1536&w=1536'
!wget -O raw_images/stock_photo2.jpg 'https://images.pexels.com/photos/1445467/pexels-photo-1445467.jpeg?auto=compress&cs=tinysrgb&dpr=3&h=1536&w=1536'

In [ ]:
import PIL.Image
img1 = PIL.Image.open('raw_images/stock_photo1.jpg')
wpercent = (256/float(img1.size[0]))
hsize = int((float(img1.size[1])*float(wpercent)))
img1 = img1.resize((256,hsize), PIL.Image.LANCZOS)
img2 = PIL.Image.open('raw_images/stock_photo2.jpg')
wpercent = (256/float(img2.size[0]))
hsize = int((float(img2.size[1])*float(wpercent)))
img2 = img2.resize((256,hsize), PIL.Image.LANCZOS)
display(img1,img2)

Now we need to get just the faces, cropped and aligned... fortunately, there's already a script for this!

In [ ]:
!python align_images.py raw_images/ aligned_images/

display(PIL.Image.open('aligned_images/stock_photo1_01.png').resize((256,256)))
display(PIL.Image.open('aligned_images/stock_photo2_01.png').resize((256,256)))

Finally, let's try encoding some images into a latent representation! That's what you came here for, right? Let's just do a quick encoding, and see how we do...

Ok, let's see how we did! Note the paths above, the generated image is in the generated_images folder, the latent representation is in the latent_representations folder, and since we generated videos of the training process, by default those are in the videos folder.

Great job, everybody, nailed it! ... wait, what's that you say, the second image doesn't quite look the same? Hmm... let's try to do a better job, shall we?

In [ ]:
!python encode_images.py aligned_images/ generated_images/ latent_representations/ \
    # --lr=0.25 --iterations=1000 --use_l1_penalty=0.5

In [ ]:
display(PIL.Image.open('generated_images/stock_photo1_01.png').resize((512, 512)))
display(PIL.Image.open('generated_images/stock_photo1_01.png').resize((512, 512)))


When in doubt, mess with the parameters... here, we ~lowered the l1 penalty on the dlatents, to allow for greater variation outside of what StyleGAN knows well, and we've lowered the learning rate and~ raised the number of iterations~, to allow for more gradual changes~. See more options below:

In [ ]:
!python encode_images.py --help

## ~ Here Be Dragons ~

Ok, that's nice, but what to do with the latents? Well, I'm glad you asked, let's see what we can do with the two latent representations we now have to rub together...

In [ ]:
import math
import pickle
import PIL.Image
import numpy as np
import config
import dnnlib
import dnnlib.tflib as tflib
from encoder.generator_model import Generator

URL_FFHQ = 'gdrive:networks/stylegan2-ffhq-config-f.pkl'
tflib.init_tf()
with dnnlib.util.open_url(URL_FFHQ) as f:
    generator_network, discriminator_network, Gs_network = pickle.load(f)

generator = Generator(Gs_network, batch_size=1, randomize_noise=False)

model_res = 1024
model_scale = int(2*(math.log(model_res,2)-1))

def generate_image(latent_vector):
    latent_vector = latent_vector.reshape((1, 18, 512))
    generator.set_dlatents(latent_vector)
    img_array = generator.generate_images()[0]
    img = PIL.Image.fromarray(img_array, 'RGB')
    return img.resize((256, 256))

def move_and_show(latent_vector, direction, coeffs):
    fig,ax = plt.subplots(1, len(coeffs), figsize=(15, 10), dpi=80)
    for i, coeff in enumerate(coeffs):
        new_latent_vector = latent_vector.copy()
        new_latent_vector[:8] = (latent_vector + coeff*direction)[:8]
        ax[i].imshow(generate_image(new_latent_vector))
        ax[i].set_title('Coeff: %0.1f' % coeff)
    [x.axis('off') for x in ax]
    plt.show()

In [ ]:
stock_photo1 = np.load('/content/stylegan2encoder/latent_representations/stock_photo2_01.npy')
stock_photo2 = np.load('/content/stylegan2encoder/latent_representations/stock_photo1_01.npy')
# Upload latent directions
# !mv /content/*.npy /content/stylegan2encoder/ffhq_dataset
# gender_direction = np.load('/content/stylegan2encoder/ffhq_dataset/gender.npy')
# age_direction = np.load('/content/stylegan2encoder/ffhq_dataset/age.npy')
# smile_direction = np.load('/content/stylegan2encoder/ffhq_dataset/smile.npy')

In [ ]:
move_and_show(stock_photo1, gender_direction, [-2, 0, 2])
move_and_show(stock_photo2, gender_direction, [-2, 0, 2])